In [3]:
import pandas as pd

In [1]:
# creating list of urls for each top5 league club from the 18-19 season
url_list = open('urls/goalkeepers_pl.txt').read().splitlines()
print(url_list[0])
print(len(url_list))

https://fbref.com/en/squads/b8fd03ef/2018-2019/matchlogs/c9/keeper/Manchester-City-Match-Logs-Premier-League
20


In [15]:
import re
import pandas as pd
import time

# empty list to store dataframes
dfs = []

# delay between requests (in seconds)
# to avoid getting blocked by the server
delay_between_requests = 2 

for url in url_list:
    try:

        tables = pd.read_html(url, header = 1)
        df = tables[0]
        
        # extract club name from the URL using regex
        club_name_match = re.search(r"squads/.*?/.*?/.*?/.*?/.*?/(.*?)-Match-Logs-Premier-League", url)
        if club_name_match:
            club_name = club_name_match.group(1).replace('-', ' ')
        else:
            raise ValueError("Club name not found in URL pattern.")
        
        # create a new column with the club name
        df['Club'] = club_name
        
        dfs.append(df)
        
        # delay before making the next request
        time.sleep(delay_between_requests)
    except Exception as e:
        print(f"Error reading data from {url}: {str(e)}")

# concatenate all dataframes into one
total_df = pd.concat(dfs, ignore_index=True)


In [24]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 760 entries, 0 to 778
Data columns (total 36 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       760 non-null    object 
 1   Time       760 non-null    object 
 2   Club       760 non-null    object 
 3   Round      760 non-null    object 
 4   Day        760 non-null    object 
 5   Venue      760 non-null    object 
 6   Result     760 non-null    object 
 7   GF         760 non-null    int64  
 8   GA         760 non-null    int64  
 9   Opponent   760 non-null    object 
 10  SoTA       760 non-null    int64  
 11  GA.1       760 non-null    int64  
 12  Saves      760 non-null    int64  
 13  Save%      733 non-null    float64
 14  CS         760 non-null    int64  
 15  PSxG       760 non-null    float64
 16  PSxG+/-    760 non-null    float64
 17  PKatt      760 non-null    int64  
 18  PKA        760 non-null    int64  
 19  PKsv       760 non-null    int64  
 20  PKm        760 

In [19]:
total_df = total_df.dropna(subset="Date")

In [21]:
club_index = total_df.columns.get_loc("Club")
club_column = total_df.pop("Club")
total_df.insert(2, "Club", club_column)

In [23]:
total_df = total_df.iloc[:, :-1]

In [25]:
total_df.to_csv("goalkeeping_stats_pl.csv", index = False)